In [12]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data

warnings.filterwarnings('ignore')
#pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
ticker = "BP.L"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [3]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1988-07-01 00:00:00+01:00,32.959076,32.959076,32.959076,32.959076,0,0.0,0.0
1988-07-04 00:00:00+01:00,32.959076,32.959076,32.959076,32.959076,0,0.0,0.0
1988-07-05 00:00:00+01:00,33.482258,33.482258,33.482258,33.482258,0,0.0,0.0
1988-07-06 00:00:00+01:00,34.397758,34.397758,34.397758,34.397758,0,0.0,0.0
1988-07-07 00:00:00+01:00,34.136169,34.136169,34.136169,34.136169,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-24 00:00:00+01:00,404.450012,410.799988,401.799988,401.799988,58531657,0.0,0.0
2024-10-25 00:00:00+01:00,401.950012,406.950012,401.149994,404.899994,24028000,0.0,0.0
2024-10-28 00:00:00+00:00,398.700012,400.100006,392.750000,399.100006,35303917,0.0,0.0


In [4]:
strategy = SmaCross
strategy.n1 = 50
strategy.n2 = 125
bt_sma = Backtest(data["2016":], SmaCross, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2016-01-04 00:00...
End                       2024-10-30 00:00...
Duration                   3222 days 00:00:00
Exposure Time [%]                   48.696043
Equity Final [$]                  11900.59149
Equity Peak [$]                  17599.838834
Return [%]                          19.005915
Buy & Hold Return [%]               79.632451
Return (Ann.) [%]                    1.991182
Volatility (Ann.) [%]               19.353668
Sharpe Ratio                         0.102884
Sortino Ratio                        0.148119
Calmar Ratio                         0.061023
Max. Drawdown [%]                  -32.630053
Avg. Drawdown [%]                   -6.106969
Max. Drawdown Duration     1462 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                    8
Win Rate [%]                             50.0
Best Trade [%]                      45.130379
Worst Trade [%]                     -9.717847
Avg. Trade [%]                    

In [5]:
bt_sma.plot()

Row(id='1450', ...)

In [6]:
%%time
stats = bt_sma.optimize(
    n1=range(20, 70, 10),
    n2=range(100, 250, 20),
    maximize='Equity Final [$]',
    constraint=lambda param: param.n1 < param.n2,
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 2.26 s, sys: 51.4 ms, total: 2.31 s
Wall time: 2.32 s


Start                     2016-01-04 00:00...
End                       2024-10-30 00:00...
Duration                   3222 days 00:00:00
Exposure Time [%]                   52.338129
Equity Final [$]                 18921.185817
Equity Peak [$]                  24859.008574
Return [%]                          89.211858
Buy & Hold Return [%]               79.632451
Return (Ann.) [%]                    7.493162
Volatility (Ann.) [%]               21.534361
Sharpe Ratio                         0.347963
Sortino Ratio                        0.540042
Calmar Ratio                         0.287186
Max. Drawdown [%]                  -26.091667
Avg. Drawdown [%]                   -4.605472
Max. Drawdown Duration      984 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                      83.654469
Worst Trade [%]                     -4.883065
Avg. Trade [%]                    

In [7]:
stats._strategy

<Strategy SmaCross(n1=30,n2=240)>

In [8]:
bt_sma.plot()

Row(id='2182', ...)

In [9]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

CPU times: user 2.84 s, sys: 33.4 ms, total: 2.87 s
Wall time: 2.88 s


Start                     1988-07-01 00:00...
End                       2024-10-30 00:00...
Duration                  13270 days 01:00:00
Exposure Time [%]                    2.511538
Equity Final [$]                    25.176142
Equity Peak [$]                       10000.0
Return [%]                         -99.748239
Buy & Hold Return [%]             1034.892218
Return (Ann.) [%]                  -14.944248
Volatility (Ann.) [%]                6.692035
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.748239
Avg. Drawdown [%]                  -99.748239
Max. Drawdown Duration    13266 days 00:00:00
Avg. Drawdown Duration    13266 days 00:00:00
# Trades                                  169
Win Rate [%]                              0.0
Best Trade [%]                      -0.894717
Worst Trade [%]                     -6.599998
Avg. Trade [%]                    

In [10]:
bt_momentum.plot()

Row(id='3008', ...)

In [11]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=True)
stats = bt_momentum.run()
stats

Start                     1988-07-01 00:00...
End                       2024-10-30 00:00...
Duration                  13270 days 01:00:00
Exposure Time [%]                    2.372008
Equity Final [$]                    24.961933
Equity Peak [$]                       10000.0
Return [%]                         -99.750381
Buy & Hold Return [%]             1034.892218
Return (Ann.) [%]                  -14.963903
Volatility (Ann.) [%]                6.544317
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.750381
Avg. Drawdown [%]                  -99.750381
Max. Drawdown Duration    13265 days 00:00:00
Avg. Drawdown Duration    13265 days 00:00:00
# Trades                                  171
Win Rate [%]                              0.0
Best Trade [%]                       -1.08908
Worst Trade [%]                     -6.392049
Avg. Trade [%]                    